In [2]:
# png,jpg専用
import io
from google.cloud import vision
from google.oauth2 import service_account

# 身元証明書のjson読み込み
credentials = service_account.Credentials.from_service_account_file('ocr.json')
client = vision.ImageAnnotatorClient(credentials=credentials)

# OCR対象の画像パス
input_file = "/Users/kii/work/python/python-study/ocr/pdf/M&Aデータ_33-62_202208/M&Aデータ_33-62_202208.pdf_000.pdf"

with io.open(input_file, 'rb') as image_file:
    content = image_file.read()
image = vision.Image(content=content)
response = client.document_text_detection(image=image)

# OCRした結果を表示
print(response.full_text_annotation.text)

f = open('test.txt', 'w')
f.write(response.full_text_annotation.text)
f.close()

In [9]:
# pdf専用
from google.cloud import vision
import pathlib

client = vision.ImageAnnotatorClient.from_service_account_json(
        'ocr.json')

# DOCUMENT_TEXT_DETECTIONを指定
feature = vision.Feature(
        type=vision.Feature.Type.DOCUMENT_TEXT_DETECTION)

curdir = 'M&Aデータ_93-98_202209'
files = list(pathlib.Path(curdir).glob('*.pdf'))

for file in files:
        # 入力
        with open(file, 'rb') as fp:
                content = fp.read()
        input_config = vision.InputConfig(
                content=content, mime_type='application/pdf')

        # リクエスト
        req = vision.AnnotateFileRequest(
                features=[feature], input_config=input_config)
        res = client.batch_annotate_files(requests=[req])

        # 1つ目のレスポンスの内容を取得（requestsを1個しか送ってないのでresponseは1つのみ）
        first_res = res.responses[0]

        # DOCUMENT_TEXT_DETECTIONではfull_text_annotationが返ってくる
        annotation = first_res.responses[0].full_text_annotation

        # 抽出されたテキストからテキストファイル作成
        file_name = str(file).replace('.pdf', '')
        file_name = str(file).replace('/' + curdir, '')

        f = open('./output/{}.txt'.format(file_name), 'w')
        f.write(annotation.text)
        f.close()

In [6]:
# プログラム1｜ライブラリ設定
import PyPDF2
import pathlib
import os

# プログラム2｜フォルダ内のPDFを全て取得
curdir = os.getcwd()
files = list(pathlib.Path(curdir).glob('*.pdf'))

# プログラム3｜フォルダ内の全てのPDFを処理
for file in files:

    # プログラム4｜分割したPDFを保管するためのフォルダ作成
    filename = file.name
    foldername = filename.replace('.pdf','')
    path = os.path.join(curdir, foldername)
    if not os.path.isdir(path):
        os.makedirs(path)

    # プログラム5｜PDFを分割
    pdf = PyPDF2.PdfFileReader(filename)
    for page in range(pdf.numPages):
        newpdf = PyPDF2.PdfFileWriter()
        newpdf.addPage(pdf.getPage(page))

        # プログラム6｜分割したPDFに名前を付けて保存
        pageNo = format(page, '0>3')
        newfilename = os.path.join(path, f'{filename}_{pageNo}.pdf')
        with open(newfilename, 'wb') as f:
            newpdf.write(f)

In [59]:
f = open('./output/a.txt', 'w')
f.write(annotation.text)
f.close()